<center><font size="16">Human Resource Analysis</font></center>

Companies want to improve and maintain a healthy relationship and environment for their employees. This can be achieved by recognizing and understanding the important factors that are associated with employee turnover and taking care of them might add into the productivity and growth of the employees. These insights can help managers for grabing the opportunity to take corrective steps to build and preserve their successful business. 


### Column Description

- satisfacion_level: Showing satisfaction of a particular employee
- last_evaluation: Showing last evaluation of a particular employee
- number_project: Showing number of projects handled a particular employee
- average_montly_hours: Showing the monthly hours that were spent the particular emloyee
- time_spend_company: Shows the number of years spent by the particular employee in the company.
- Work_accident: Showing an employee has whether been part of the company or not.
- left: Tells either and employee has left the company or not. Shows two values 0= not left, 1= left
- promotion_last_5years: Shows that the whether the employee has got any promotion in the last 5 years or not.
- dept: Shows the departments
- salary: Shows the salary type of the employee

# Wrangling & EDA

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%matplotlib inline

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

## 2. Loading Data & Basic Analysis

In [ ]:
data= pd.read_csv('../input/human-resource/HR_comma_sep.csv')
df=data.copy()

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.columns = df.columns.str.replace("sales", "dept")


In [ ]:
df.dtypes

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

# 3. Exploration

Before moving ahead, let us check the details of different variables in the data

In [ ]:
sns.countplot(x='left', data=df);
#or

#df[df['left']==1]['left'].sum()

out of 14999, 3571 people left

In [ ]:
sns.countplot(x='number_project', data=df);

more than 4500 employees were assigned 4 projects, around 4000 employees were assigned 3 projects.
Least amount of employees were assinged 7 projects

In [ ]:
dfgroup=df.groupby('number_project')['left'].sum().reset_index(name='left')
dfgroup

In [ ]:
sns.barplot(x='number_project', y='left', data=dfgroup);

more than 1500 people left with assigned project number 2

One of the reason might be that they weren't able to do it or didn't have the skills to.

## **Time spent at the company**

In [ ]:
sns.countplot(x='time_spend_company', data=df);

maximum number of time spend by the employees are 3 years

In [ ]:
dftemp = df.groupby('number_project')['left'].value_counts(normalize=True).reset_index(name='Perc')
dftemp[dftemp['left']==1][['number_project','Perc']]

In [ ]:
temp = df.groupby('time_spend_company')['left'].value_counts(normalize=True).reset_index(name='Perc')

sns.barplot(x='time_spend_company', y='Perc', hue='left', data=temp[temp['left']==1]);

more than 55% chance to leave the company after five years of service.

## **Employees engaged in any work accident**

In [ ]:
# Number of employees involved in work accident
sns.countplot(x='Work_accident', data=df);

more than 2150 employees were engaged in a work accident out of which only less than 200 left the company.


In [ ]:
sns.countplot(x='Work_accident',hue='left', data=df[df['Work_accident']==1]);

## **Promotions in last 5 years**

In [ ]:
sns.countplot(x='promotion_last_5years', hue='left', data=df[df['promotion_last_5years']==1]);

more than 300 employees got promoted after five years and less than 20 employees left after being promoted.

## **Salary trends**

In [ ]:
sns.countplot(data=df,x='salary');

7000+ employees are being paid low and 6000+ are being paid medium salary. less than 1500 are high paid

In [ ]:
df_temp = df.groupby('salary')['left'].value_counts(normalize=True).reset_index(name='Perc')
df_temp[df_temp['left']==1]

In [ ]:
sns.barplot(x='salary', y='Perc', data=df_temp[df_temp['left']==1]);

almost 30% of low salaried employees are most likely to leave

## **Employees per Department**

In [ ]:
plt.figure(figsize=(15,6));
sns.countplot(x='dept', data=df);

sales department has maximum number of employees

In [ ]:
temp = df.groupby('dept')['left'].value_counts(normalize=True).reset_index(name='Perc')

In [ ]:
plt.figure(figsize=(15,6));

sns.barplot(x='dept', y='Perc', hue='left', data=temp[temp['left']==1]);

HR department has highest percentage of turn-over

## **Satisfaction Level**

In [ ]:
sns.kdeplot(df[df['left'] == 0]['satisfaction_level'], shade=True,label="not left");
sns.kdeplot(df[df['left'] == 1]['satisfaction_level'], shade=True,label="left");

50 to 100 percent of satifaction of those who are employeed. 10% and mostly 40% satisfaction of those employees who left.

# Feature Engineering

For feature engineering we will two new features. Looking at the the satisfcation we can conclude that people who are leaving have a low satisfaction level, most likely below 0.5 are leaving and people having a high satisfaction_level, most likely above 0.5 are likely to stay. 

In [ ]:
df['satisfaction_level_type']=''
df.loc[ df['satisfaction_level'] >= 0.5, 'satisfaction_level_type'] = 'High'
df.loc[ df['satisfaction_level'] < 0.5, 'satisfaction_level_type'] = 'Low'

In [ ]:
sns.countplot(x='satisfaction_level_type', hue='left', data=df);

Previously we saw that employees having high number of projects are leaving. We also saw that some employees with extremely less number of projects are also leaving the company. Let us see how number of projects and satisfaction level are related.<br>
We can see this by checking the satisfaction level type and number of projects in according to that specific type. 

In [ ]:
sns.countplot(x='number_project', hue='satisfaction_level_type', data=df);

Our prediction was true. Employees having high number of projects such as 6 and 7, and employees having extremely less number of projects are leaving the company because of low satisfaction level.

Now lets check number of projects for low satisfaction level

In [ ]:
sns.countplot(x='number_project', data=df[df['satisfaction_level_type']=='Low']);

Notice that there is are people associated with low satisfaction level, have either too less projects, i.e. number of projects = 2 or high number of projects i.e. 6 or even 7. The over all count for 7 is low but there is a significant raise if we compare its count with higher satisfaction level.

In [ ]:
df['employee_type']=''

In [ ]:
df.loc[ df['number_project'] == 2, 'employee_type'] = 'unburdened'
df.loc[ (df['number_project'] >= 3) & (df['number_project'] <= 5), 'employee_type'] = 'Satisfactory'
df.loc[ df['number_project'] >= 6, 'employee_type'] = 'Burdened'

In [ ]:
sns.countplot(x='employee_type', hue='left', data=df);

### Remove Redundant Features

In [ ]:
df.drop(['number_project', 'satisfaction_level'] ,axis=1,inplace=True)
df

# Machine Learning

Before moving further, we need to apply one-hot encoding on categorical variables i.e. **dept, salary, satisfaction_level_type,** and **employee_type**

In [ ]:
df.head()

In [ ]:
df_OneHot=pd.get_dummies(df,columns=['dept', 'salary','satisfaction_level_type','employee_type'])
df_OneHot.head()
dfoh = df_OneHot.copy()

In [ ]:
dfoh

In [ ]:
X=dfoh.drop(['left'], axis=1)
Y=dfoh[['left']]

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
xtrain, xtest, ytrain, ytest = train_test_split(X, Y, test_size=0.30, random_state=100, shuffle=True)

In [ ]:
print(xtrain.shape, ytrain.shape)
print(xtest.shape,ytest.shape)

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
clf_rf = RandomForestClassifier(random_state=100)
clf_rf.fit(xtrain, ytrain)

In [ ]:
rf_pred=clf_rf.predict(xtest)
rf_pred_prb = clf_rf.predict_proba(xtest)[:, 1]

**Precision**

In [ ]:
from sklearn.metrics import precision_score

In [ ]:
rf_precion= precision_score(ytest, rf_pred)
print("Precision: {}".format(rf_precion))

**Accuracy**

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
rf_acc=accuracy_score(ytest, rf_pred)
print("Accuracy: {}".format(rf_acc))

**Recall**

In [ ]:
from sklearn.metrics import recall_score

In [ ]:
rf_recall=recall_score(ytest, rf_pred)
print("Recall: {}".format(rf_recall))

**F1-Score**

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
rf_f1_score=f1_score(ytest, rf_pred)
print("F1 Score: {}".format(rf_f1_score))

**Classification Report**

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(ytest, rf_pred))

## Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
lr_model = LogisticRegression(random_state=100, solver='liblinear')
lr_model.fit(xtrain, ytrain)

In [ ]:
lr_pred = lr_model.predict(xtest)

**Accuracy**

In [ ]:
lr_accuracy= accuracy_score(ytest, lr_pred)
print("Accuracy: {}".format(lr_accuracy))

**Precision**

In [ ]:
lr_precision = precision_score(ytest, lr_pred)
print("Precision: {}".format(lr_precision))

**Recall**

In [ ]:
lr_recall= recall_score(ytest, lr_pred)
print("Accuracy: {}".format(lr_recall))

**F1 Score**

In [ ]:
lr_f1_score=f1_score(ytest, lr_pred)
print("Accuracy: {}".format(lr_f1_score))

**Classification Report**

In [ ]:
print(classification_report(ytest, lr_pred))

## Ada Boost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
clf_adb = AdaBoostClassifier(random_state=100)
clf_adb.fit(xtrain, ytrain)

In [ ]:
pred_clf_adb=clf_adb.predict(xtest)

**Accuracy**

In [ ]:
ab_accuracy= accuracy_score(ytest, pred_clf_adb)
print("Accuracy: {}".format(ab_accuracy))

**Precision**

In [ ]:
ab_precision=precision_score(ytest, pred_clf_adb)
print("Precision: {}".format(ab_precision))

**Recall**

In [ ]:
ab_recall=recall_score(ytest, pred_clf_adb)
print("Recall: {}".format(ab_recall))

**F1-Score**

In [ ]:
ab_f1_score=f1_score(ytest, pred_clf_adb)
print("F1-score: {}".format(ab_f1_score))

**Classification Report**

In [ ]:
print(classification_report(ytest, pred_clf_adb))

## XG Boost

In [ ]:
import xgboost as xgb

In [ ]:
clf_xgb = xgb.XGBClassifier(seed=25,nthread=1,random_state=100)
clf_xgb.fit(xtrain, ytrain)

In [ ]:
pred_clf_xgb= clf_xgb.predict(xtest)

**Accuracy**

In [ ]:
xgb_accuracy=accuracy_score(ytest, pred_clf_xgb)
print("Accuracy: {}".format(xgb_accuracy))

**Precision**

In [ ]:
xgb_precision=precision_score(ytest, pred_clf_xgb)
print("Precision: {}".format(xgb_precision))

**Recall**

In [ ]:
xgb_recall=recall_score(ytest, pred_clf_xgb)
print("Recall: {}".format(xgb_recall))

**F1-Score**

In [ ]:
xgb_f1_score=f1_score(ytest, pred_clf_xgb)
print("F1-score: {}".format(xgb_f1_score))

**Classification Report**

In [ ]:
print(classification_report(ytest,pred_clf_xgb))

# Result Comparisons

In [ ]:
comparison_dict={"Algorithm":["Random Forest","Logistic Regression","XGBoost","Ada Boost"],
                 "Accuracy":[rf_acc,lr_accuracy,xgb_accuracy,ab_accuracy],
                 "Precision":[rf_precion,lr_precision,xgb_precision,ab_precision],
                 "Recall":[rf_recall,lr_recall,xgb_recall,ab_recall],
                 "F1 Score":[rf_f1_score,lr_f1_score,xgb_f1_score,ab_f1_score]
                }

In [ ]:
comparison = pd.DataFrame(comparison_dict)
comparison.sort_values(['Recall', 'Accuracy'], ascending=False)

I don't believe that there is a need for tunning as Random Forrest is giving us the best results so far

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid1 = {"n_estimators": [9, 18, 27, 36, 45, 54, 63],
               "max_depth": [1, 5, 10, 15, 20, 25, 30],
               "min_samples_leaf": [1, 2, 4, 6, 8, 10]}

RF = RandomForestClassifier(random_state=100)

RF_cv1 = GridSearchCV(RF, param_grid1, cv=5, scoring='accuracy', n_jobs=4)


RF_cv1.fit(xtrain, ytrain)

RF_cv1.best_params_, RF_cv1.best_score_

"accuracy: 0.9848 > 0.9825, so there is no need for tuning"